<h1>Presentation of the project of the indexation of files </h1>

In [ ]:
from os import listdir
from os.path import isfile, join
from ipywidgets import FloatProgress
from IPython.display import display
from bs4 import BeautifulSoup as bs
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import nltk
import string
import operator
import shutil,os
import re
import sys
%load_ext autoreload
%autoreload 1
%aimport util_index
%aimport util_posting
%aimport fagin
%aimport graph

In [ ]:
WRITING_PATH_POSTING_LIST = "../data/saveFile/"
WRITING_PATH_UTIL = "../data/util/"
NAME_DOC_LIST = "docList"
NB_DOCUMENT = 1000

<h1>Creation of the index File</h1>

First we have to clean the repository of all old version of the algorithm.

In [ ]:
util_index.cleanRepository(WRITING_PATH_POSTING_LIST)

In [ ]:
vocList = {}
docLenght = {}
util_index.buildIndexFile(vocList, docLenght, WRITING_PATH_POSTING_LIST, NB_DOCUMENT)

writing in file, the format is : "docId" "nombre Of word contained in the doc"

In [ ]:
util_index.writingDictInFile(docLenght, WRITING_PATH_UTIL, NAME_DOC_LIST, " ")

<h1> Creation Posting List</h1>

In [ ]:
util_posting.createPostingList()

<h1>Graph Generation</h1>

In [ ]:
nbCuts = [10, 50, 100, 500, 1000, 5000, 10000, 50000]
result = graph.computeDictRelationWithSizeTime(nbCuts,WRITING_PATH_POSTING_LIST, WRITING_PATH_UTIL, NAME_DOC_LIST)
graph.displayResult( result,
                    'trade-off size document / time needed to build posting list', 
                    'log(mean size document in bytes)',
                    'time in second')

In [ ]:
def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

In [ ]:
class PostingList(object):
    # Args :
    # qt is a string containing the query term this PL is made for
    # ordered_list is a list of (score, doc_id) ordered in decreasing score
    # access_dict (optional, can be computed from ordered_list) is a dict associating a doc to its score in this PL
    def __init__(self, qt, ordered_list, access_dict=None):
        self.qt=qt
        self.ordered_list = ordered_list
        if access_dict is not None:
            self.access_dict = access_dict
        else:
            self.access_dict = {}
            for score,doc in ordered_list:
                assert doc not in self.access_dict
                self.access_dict[doc] = score

        self.docs_visited = set()
        self.ordered_idx = 0

    # Returns : A (score, doc_index) tuple corresponding to the first non-visited entry in the ordered traversal
    def seek_next(self):
        while self.ordered_list[self.ordered_idx][1] in self.docs_visited:
            self.ordered_idx += 1
        return self.ordered_list[self.ordered_idx]

    # Returns : The score of the item preceding the next ordered accessed item
    def next_item_predecessor_score(self):
        tmp_idx = self.ordered_idx
        while self.ordered_list[tmp_idx][1] in self.docs_visited:
            self.ordered_idx += 1
        return self.ordered_list[tmp_idx-1][0]

    # Args :
    # doc_id is an integer containing the id of the document we want to mark as visited in the sorted access
    def mark_visited(self, doc_id):
        assert doc_id not in self.docs_visited
        self.docs_visited.add(doc_id)

    # Args :
    # doc_id is an integer containing the document id to lookup in the random access
    #
    # Returns : The score of the queried document in the PL
    def random_lookup(self, doc_id):
        return self.access_dict[doc_id]

#TODO(mathishammel): Use a real priority queue. Lists are disgusting
class TopEntries(object):
    def __init__(self, k):
        self.k = k
        self.top = []

    def insert(self, priority, element):
        self.top += [(priority, element)]
        self.top = sorted(self.top, reverse=True)[:self.k]

    def pop_lowest(self):
        res = self.top[-1]
        del self.top[-1]
        return res

    def get_min_score(self):
        if len(self.top) == 0:
            return -1.0
        return self.top[-1][0]

def calc_avg(lst):
    return float(sum(lst))/len(lst)

# Args :
# posting_lists is a list of PostingList objects corresponding to the PLs for all query terms.
# k is an integer containing the length of the desired top-k ranking
#
# Returns : A list containing the (total_score, doc_id) for the top-k elements
def top_k_thresh(posting_lists, k, epsilon=0.0):
    #Check if we're not trying to get an impossible top-k
    for posting_list in posting_lists:
        assert k < len(posting_list.ordered_list)

    top_k = TopEntries(k)
    top_non_visited = []
    for posting_list in posting_lists:
        top_non_visited.append(posting_list.seek_next())

    eprint('Initialized top inorder array :', top_non_visited)
    threshold = 1e999 # Close enough to infinity, hopefully...
    
    while top_k.get_min_score() < threshold / (1.0 + epsilon):
        eprint('Starting new round')
        selected_idx = -1
        best_indiv_in_order = -1.0 # Assuming all PL scores are positive
        for idx, score in enumerate(top_non_visited):
            if best_indiv_in_order < score:
                selected_idx = idx
                best_indiv_in_order = score
        selected_element = top_non_visited[selected_idx]
        selected_score, selected_doc_id = selected_element
        eprint('  Selected PL index is', selected_idx)
        eprint('  Selected element is', selected_element)
        posting_lists[selected_idx].mark_visited(top_non_visited[selected_idx][1])
        top_non_visited[selected_idx] = posting_lists[selected_idx].seek_next()

        scores = []
        for idx in range(len(posting_lists)):
            if idx == selected_idx:
                scores.append(selected_score)
                continue
            scores.append(posting_lists[idx].random_lookup(selected_doc_id))
            posting_lists[idx].mark_visited(selected_doc_id)
            if top_non_visited[idx][1] == selected_doc_id:
                top_non_visited[idx] = posting_lists[idx].seek_next()
        
        eprint('  Individual scores for document',selected_doc_id,'are', scores)
        tot_score = calc_avg(scores)
        eprint('  Average score is', tot_score)
        top_k.insert(tot_score, selected_doc_id)
        eprint('  Current top K is', top_k.top)

        all_lists_ready = True
        next_prev_scores = []
        for idx,posting_list in enumerate(posting_lists):
            if posting_list.ordered_list[0][1] not in posting_list.docs_visited:
                eprint('  Posting list',idx,'is not ready for threshold computation yet, aborting.')
                all_lists_ready = False
                break
            next_prev_scores.append(posting_list.next_item_predecessor_score())
        eprint('  Nextprev scores are',next_prev_scores)
        threshold = calc_avg(next_prev_scores)
    return top_k.top

In [ ]:
#Test Fagin's 2 on ppt example
pl1 = PostingList('hello', [(0.9,2),(0.8,5),(0.7,6),(0.6,4),(0.5,1),(0.4,3)])
pl2 = PostingList('world', [(0.85,3),(0.8,5),(0.75,2),(0.74,6),(0.74,1),(0.7,4)])

print top_k_thresh([pl1, pl2], 3)

<h1>Fagins</h1>

In [ ]:
posting_lists = util_posting.creat_posting_list_obj_list("cat dog")

In [ ]:
resultFagin = fagin.top_k_thresh(posting_lists, 4, epsilon=0.0)

In [ ]:
print(resultFagin)